In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Read in Data from our csv files
PriceData = pd.read_csv('../Data/global_food_prices.csv', low_memory=False)
CorruptData = pd.read_csv('../Data/Corruption_Index.csv', low_memory=False)
Categories = pd.read_csv('../Data/FoodTypes.csv', low_memory=False)

In [3]:
#get only the columns we need
PriceData = PriceData.filter(regex='adm0_name|cm_name|mp_price|mp_month|mp_year') #|mp_price|mp_year
#remove the hypen and whitecpaces so we can get our real item name
PriceData['cm_name'] = PriceData['cm_name'].str.split(' ').str.get(0)
PriceData['cm_name'] = PriceData['cm_name'].str.split('-').str.get(0)
#make our countries and items lowercase
PriceData['cm_name'] = PriceData['cm_name'].str.lower()
PriceData['adm0_name'] = PriceData['adm0_name'].str.lower()
MyPriceData.reset_index()

NameError: name 'MyPriceData' is not defined

In [ ]:
#grab our food items onion, cucumber, potatoes, garlic for our countries jordan, turkey, colombia, ecuador
onionDF = PriceData[PriceData['cm_name'].str.contains("onions")]
onionDF = onionDF[onionDF['adm0_name'].isin(['colombia', 'ecuador', 'jordan', 'turkey'])]
onionGrouped = onionDF.groupby(['adm0_name', 'cm_name', 'mp_year'], as_index=False)['mp_price'].mean()
print(onionGrouped.head(5))

cucuDF = PriceData[PriceData['cm_name'].str.contains("cucumbers")]
cucuDF = cucuDF[cucuDF['adm0_name'].isin(['colombia', 'ecuador', 'jordan', 'turkey'])]
cucuGrouped = cucuDF.groupby(['adm0_name', 'cm_name', 'mp_year'], as_index=False)['mp_price'].mean()
print(cucuGrouped.head(5))

potDF = PriceData[PriceData['cm_name'].str.contains("potatoes")]
potDF = potDF[potDF['adm0_name'].isin(['colombia', 'ecuador', 'jordan', 'turkey'])]
potGrouped = potDF.groupby(['adm0_name', 'cm_name', 'mp_year'], as_index=False)['mp_price'].mean()
print(potGrouped.head(5))

garlicDF = PriceData[PriceData['cm_name'].str.contains("garlic")]
garlicDF = garlicDF[garlicDF['adm0_name'].isin(['colombia', 'ecuador', 'jordan', 'turkey'])]
garlicGrouped = garlicDF.groupby(['adm0_name', 'cm_name', 'mp_year'], as_index=False)['mp_price'].mean()
print(garlicGrouped.head(5))

#combine all our datasets into one
MyPriceData = pd.concat([onionGrouped, cucuGrouped, potGrouped, garlicGrouped])
print(MyPriceData.head(5))

#include years of concern
MyPriceData = MyPriceData.loc[(MyPriceData['mp_year'] > 2012) & (MyPriceData['mp_year'] < 2021)]
print(MyPriceData.head(5))

In [ ]:
#for the cpi index read in only the countires we want
CorruptData = CorruptData[CorruptData['Country'].isin(['Colombia', 'Ecuador', 'Jordan', 'Turkey'])]
#filter in  the cpi years that we want 2013-2020
CorruptData = CorruptData.filter(regex='2013|2014|2015|2016|2017|2018|2019|2020|Country')
#Get only the columns we need. all we care about is country and cpi
CorruptData = CorruptData.filter(regex='Country|CPI')
#Correct Indexes
CorruptData = CorruptData.reset_index()
#Make header years only
CorruptData = CorruptData.rename(columns = lambda x: x.replace('CPI score ', ''))
CorruptData = CorruptData.rename(columns = lambda x: x.replace('CPI Score ', ''))
CorruptData = CorruptData.rename(columns = lambda x: x.replace(' ', ''))
#Put all values to lower case
Categories = Categories.applymap(lambda s: s.lower() if type(s) == str else s)

In [ ]:
#we need to convert our cpi to percentges so they can graph better along the y axis. item prices can vary greatly so lets use percentages  instead
columns = ['index', 'country', 'date', 'inflation_score']

corruptTransform = pd.DataFrame({
                   'date': pd.Series(dtype='str'),
                   'inflation_score': pd.Series(dtype='float')})

for index, row in CorruptData.iterrows():
    ctRow20 = {'Country': row['Country'], 'date': '2020', 'inflation_score': row['2020']}
    ctRow19 = {'Country': row['Country'], 'date': '2019', 'inflation_score': row['2019']}
    ctRow18 = {'Country': row['Country'], 'date': '2018', 'inflation_score': row['2018']}
    ctRow17 = {'Country': row['Country'], 'date': '2017', 'inflation_score': row['2017']}
    ctRow16 = {'Country': row['Country'], 'date': '2016', 'inflation_score': row['2016']}
    ctRow15 = {'Country': row['Country'], 'date': '2015', 'inflation_score': row['2015']}
    ctRow14 = {'Country': row['Country'], 'date': '2014', 'inflation_score': row['2014']}
    ctRow13 = {'Country': row['Country'], 'date': '2013', 'inflation_score': row['2013']}
    corruptTransform = corruptTransform.append(ctRow20, ignore_index = True)
    corruptTransform = corruptTransform.append(ctRow19, ignore_index = True)
    corruptTransform = corruptTransform.append(ctRow18, ignore_index = True)
    corruptTransform = corruptTransform.append(ctRow17, ignore_index = True)
    corruptTransform = corruptTransform.append(ctRow16, ignore_index = True)
    corruptTransform = corruptTransform.append(ctRow15, ignore_index = True)
    corruptTransform = corruptTransform.append(ctRow14, ignore_index = True)
    corruptTransform = corruptTransform.append(ctRow13, ignore_index = True)
    
#this function will find the year over year increate in cpi per country per year
def findYtYPriceChange(row):
    nextYear = int(row['date']) - 1
    matchMax = np.where((corruptTransform['Country'] == row['Country']) & (corruptTransform['date'].astype(str) == str(nextYear)))
    percentChangeMaxYear = 0
    if matchMax[0].size > 0:
        rowFinds = corruptTransform.loc[matchMax].reset_index()
        rowFind = rowFinds.iloc[0]
        percentChangeMaxYear = (((row['inflation_score']- rowFind['inflation_score']) / rowFind['inflation_score']) * 100)
    return percentChangeMaxYear
corruptTransform['CPI_Year_To_Year_Price_Change_%'] = corruptTransform.apply(lambda x: findYtYPriceChange(x), axis = 1)
    
corruptTransform

#cleaning up our data for later like lowercase countries, rename columns to match other data sets, converting years from string to int
corruptTransform['Country'] = corruptTransform['Country'].str.lower()
corruptTransform.rename(columns = {'Country':'adm0_name', 'date':'mp_year'}, inplace = True)
corruptTransform['mp_year']=corruptTransform['mp_year'].astype(int)
corruptTransform.fillna(0)

In [ ]:
#all our data is in the format we need so lets start graphing
#the x axis will be the yeats
#the y xis will be the average price for my fruits/veggies categories
#notice in the last chart where everything is combined. colombia prices in our dataset had really high values so the rest of the countries look flat in comparison
#we will later fix this by normalizing the y axis to use increase in price percentages instead of average price
c = MyPriceData[(MyPriceData['adm0_name'] =='colombia')]
c = c.reset_index()
cG = c.groupby([ 'mp_year'], as_index=False)['mp_price'].mean()
cP = cG.plot(x='mp_year', y='mp_price', title = 'Price Veggies/Fruit In Colombia', legend='False')
cP.set_xlabel("Year")
cP.set_ylabel("Average Price in $")
cP

e = MyPriceData[(MyPriceData['adm0_name'] =='ecuador')]
eG = e.groupby([ 'mp_year'], as_index=False)['mp_price'].mean()
eP = eG.plot(x='mp_year', y='mp_price', title = 'Price Veggies/Fruit In Ecuador', legend='False')
eP.set_xlabel("Year")
eP.set_ylabel("Average Price in $")
eP

j = MyPriceData[(MyPriceData['adm0_name'] =='jordan')]
jG = j.groupby([ 'mp_year'], as_index=False)['mp_price'].mean()
jP = jG.plot(x='mp_year', y='mp_price', title = 'Price Veggies/Fruit In Jordan', legend='False')
jP.set_xlabel("Year")
jP.set_ylabel("Average Price in $")
jP

t = MyPriceData[(MyPriceData['adm0_name'] =='turkey')]
t = t.reset_index()
tG = t.groupby([ 'mp_year'], as_index=False)['mp_price'].mean()
tP = tG.plot(x='mp_year', y='mp_price', title = 'Price Veggies/Fruit In Turkey', legend='False')
tP.set_xlabel("Year")
tP.set_ylabel("Average Price in $")
tP

#lets check the average of all prices for all countries 
allCountries = MyPriceData.groupby([ 'mp_year'], as_index=False)['mp_price'].mean()
allCountriesPlot = allCountries.plot(x='mp_year', y='mp_price', title = 'Price Veggies/Fruit Of All Countries', legend='False')
allCountriesPlot.set_xlabel("Year")
allCountriesPlot.set_ylabel("Average Price in $")
allCountriesPlot

ax = cG.plot(x='mp_year', y='mp_price')
ax = eG.plot(ax=ax, x='mp_year', y='mp_price')
ax = jG.plot(ax=ax, x='mp_year', y='mp_price')
ax = tG.plot(ax=ax, x='mp_year', y='mp_price')
ax.set_xlabel("Year")
ax.set_ylabel("Average Price $")
ax.legend(["Colombia", "Ecuador", "Jordan", "Turkey"], prop={'size': 10});

In [ ]:
#this function will again create a new column to show the % increase of prices. this is to normalize our y axis so we can later graph the cpi which we already converted to % increase  
def percentChangeDebug(row):
    nextYear = int(row['mp_year']) - 1
    #matchMax = np.where((MyPriceData['adm0_name'] == MyPriceData['adm0_name']) & (MyPriceData['cm_name'] == row['cm_name']) & (MyPriceData['mp_year'].astype(str) == str(nextYear)))
    subQu = MyPriceData[(MyPriceData['adm0_name'] == MyPriceData['adm0_name']) & (MyPriceData['cm_name'] == row['cm_name']) & (MyPriceData['mp_year'].astype(str) == str(nextYear))]
    subQF = MyPriceData[(MyPriceData['adm0_name'] == row['adm0_name'])]
    subQS = subQF[(subQF['cm_name'] == row['cm_name'])]
    subQT = subQS[(subQS['mp_year'].astype(str) == str(nextYear))]
    #print(subQT)
    percentChangeMaxYear = 0
    if len(subQT) > 0:
        #rowFind = subQu.iloc[0]
        rowFind = subQT.iloc[0]
        #print('nextYear', nextYear)
        #print(row,'->')
        #print(rowFind, '<-')
        percentChangeMaxYear = (((row['mp_price']- rowFind['mp_price']) / rowFind['mp_price']) * 100)
        #print(row['mp_price']- rowFind['mp_price'])
        #print('%%%%%->%%%',percentChangeMaxYear)    
    return percentChangeMaxYear
        
MyPriceData['Year_To_Year_Price_Change_%'] = MyPriceData.apply(lambda x: percentChangeDebug(x), axis = 1)
MyPriceData

In [ ]:
#we have normalized our data to use a % scale for both prices and corrruptioin index now when we plot everything should be clearer
#x axis is years
#y axis is % increase in prices year over year
#the last chart is all 4 charts put together in one chart. here you can clearly see that while not perfect most countries follow the same pattern. when prices increase in one country they tend to increase in others

cC = MyPriceData[(MyPriceData['adm0_name'] =='colombia')]
cC = cC.reset_index()
cCG = (cC.groupby(['adm0_name', 'mp_year'], as_index=False)['Year_To_Year_Price_Change_%'].mean()) 
cCG.fillna(0)
cPlot = cCG.plot(x='mp_year', y='Year_To_Year_Price_Change_%', title = 'Price Veggies/Fruit In Colombia', legend='False')
cPlot.set_xlabel("Year")
cPlot.set_ylabel("Year to Year Change in %")

eC = MyPriceData[(MyPriceData['adm0_name'] =='ecuador')]
eC = eC.reset_index()
eCG = (eC.groupby(['adm0_name', 'mp_year'], as_index=False)['Year_To_Year_Price_Change_%'].mean()) 
eCG.fillna(0)
ePlot = eCG.plot(x='mp_year', y='Year_To_Year_Price_Change_%', title = 'Price Veggies/Fruit In Equador', legend='False')
ePlot.set_xlabel("Year")
ePlot.set_ylabel("Year to Year Change in %")

jC = MyPriceData[(MyPriceData['adm0_name'] =='jordan')]
jC = jC.reset_index()
jCG = (jC.groupby(['adm0_name', 'mp_year'], as_index=False)['Year_To_Year_Price_Change_%'].mean()) 
jCG.fillna(0)
jPlot = jCG.plot(x='mp_year', y='Year_To_Year_Price_Change_%', title = 'Price Veggies/Fruit In Jordan', legend='False')
jPlot.set_xlabel("Year")
jPlot.set_ylabel("Year to Year Change in %")

tC = MyPriceData[(MyPriceData['adm0_name'] =='turkey')]
tC = tC.reset_index()
tCG = (tC.groupby(['adm0_name', 'mp_year'], as_index=False)['Year_To_Year_Price_Change_%'].mean()) 
tCG.fillna(0)
tPlot = tCG.plot(x='mp_year', y='Year_To_Year_Price_Change_%', title = 'Price Veggies/Fruit In Turkey', legend='False')
tPlot.set_xlabel("Year")
tPlot.set_ylabel("Year to Year Change in %")

ax = cCG.plot(x='mp_year', y='Year_To_Year_Price_Change_%')
ax = eCG.plot(ax=ax, x='mp_year', y='Year_To_Year_Price_Change_%', title = 'Price Fluctuations %')
ax = jCG.plot(ax=ax, x='mp_year', y='Year_To_Year_Price_Change_%')
ax = tCG.plot(ax=ax, x='mp_year', y='Year_To_Year_Price_Change_%')
ax.set_xlabel("Year")
ax.set_ylabel("Year to Year Change in %")
ax.legend(["Colombia", "Ecuador", "Jordan", "Turkey"], prop={'size': 10});

In [ ]:
#lets graph our cpi per country below.
#the x axis is the year
#the y axis is the % increase year over year in cpi score
#the last chart is all 4 charts put together in one chart. here you can clearly see that while not perfect most countries follow the same pattern. when cpi increase in one country they tend to increase in others

cpiC = corruptTransform[(corruptTransform['adm0_name'] =='colombia')]
cpiCG =  (cpiC.groupby(['adm0_name', 'mp_year'], as_index=False)['CPI_Year_To_Year_Price_Change_%'].mean()) 
cpiCPlot = cpiCG.plot(x='mp_year', y='CPI_Year_To_Year_Price_Change_%', title = 'CPI In Colombia', legend='False')
cpiCPlot.set_xlabel("Year")
cpiCPlot.set_ylabel("Year to Year Change in %")

cpiE = corruptTransform[(corruptTransform['adm0_name'] =='ecuador')]
cpiEG =  (cpiE.groupby(['adm0_name', 'mp_year'], as_index=False)['CPI_Year_To_Year_Price_Change_%'].mean()) 
cpiEPlot = cpiEG.plot(x='mp_year', y='CPI_Year_To_Year_Price_Change_%', title = 'CPI In Ecuador', legend='False')
cpiEPlot.set_xlabel("Year")
cpiEPlot.set_ylabel("Year to Year Change in %")

cpiJ = corruptTransform[(corruptTransform['adm0_name'] =='jordan')]
cpiJG =  (cpiJ.groupby(['adm0_name', 'mp_year'], as_index=False)['CPI_Year_To_Year_Price_Change_%'].mean()) 
cpiJPlot = cpiJG.plot(x='mp_year', y='CPI_Year_To_Year_Price_Change_%', title = 'CPI In Jordan', legend='False')
cpiJPlot.set_xlabel("Year")
cpiJPlot.set_ylabel("Year to Year Change in %")

cpiT = corruptTransform[(corruptTransform['adm0_name'] =='turkey')]
cpiTG =  (cpiT.groupby(['adm0_name', 'mp_year'], as_index=False)['CPI_Year_To_Year_Price_Change_%'].mean()) 
cpiTPlot = cpiTG.plot(x='mp_year', y='CPI_Year_To_Year_Price_Change_%', title = 'CPI In Turkey', legend='False')
cpiTPlot.set_xlabel("Year")
cpiTPlot.set_ylabel("Year to Year Change in %")


ax = cpiCG.plot(x='mp_year', y='CPI_Year_To_Year_Price_Change_%')
ax = cpiEG.plot(ax=ax, x='mp_year', y='CPI_Year_To_Year_Price_Change_%', title = 'CPI Fluctuations %')
ax = cpiJG.plot(ax=ax, x='mp_year', y='CPI_Year_To_Year_Price_Change_%')
ax = cpiTG.plot(ax=ax, x='mp_year', y='CPI_Year_To_Year_Price_Change_%')
ax.set_xlabel("Year")
ax.set_ylabel("Year to Year Change in CPI %")
ax.legend(["Colombia", "Ecuador", "Jordan", "Turkey"], prop={'size': 10});


In [ ]:
#lets combine our food price data set and our cpi data set and graph them together
#x axis is the yearrs
#y axis is the %change in ...
#we have two lines. the cpi line and the categories price line. they are both using % increase and we can see some correlation in cpi and price increase percentages

mergeC = pd.merge(cpiCG, cCG, on=['adm0_name','mp_year'], how='left')
ax = mergeC.plot(x='mp_year', y=['CPI_Year_To_Year_Price_Change_%','Year_To_Year_Price_Change_%'], title = 'CPI Score vs Price for Colombia')
ax.set_xlabel("Year")
ax.set_ylabel("% Change")


mergeE = pd.merge(cpiEG, eCG, on=['adm0_name','mp_year'], how='left')
ax = mergeE.plot(x='mp_year', y=['CPI_Year_To_Year_Price_Change_%','Year_To_Year_Price_Change_%'], title = 'CPI Score vs Price for Equador')
ax.set_xlabel("Year")
ax.set_ylabel("% Change")

mergeJ = pd.merge(cpiJG, jCG, on=['adm0_name','mp_year'], how='left')
ax = mergeJ.plot(x='mp_year', y=['CPI_Year_To_Year_Price_Change_%','Year_To_Year_Price_Change_%'], title = 'CPI Score vs Price for Jordan')
ax.set_xlabel("Year")
ax.set_ylabel("% Change")

mergeT = pd.merge(cpiTG, tCG, on=['adm0_name','mp_year'], how='left')
ax = mergeT.plot(x='mp_year', y=['CPI_Year_To_Year_Price_Change_%','Year_To_Year_Price_Change_%'], title = 'CPI Score vs Price for Turkey')
ax.set_xlabel("Year")
ax.set_ylabel("% Change")